In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/stat940project')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import os
import numpy as np
import random
import math
import torch.optim as optim
from tqdm.auto import tqdm, trange

In [ ]:
from dataset import load_data, save_data, load_targets, AdversarialDataset, load_adv_data
from utils.augmentations import *
from utils.utils import run, set_seed, pcshow
from models import Pointnet, DGCNN
from attacks import FGM, IFGM

In [ ]:
set_seed(1)

In [ ]:
parent_dir = '/content/drive/MyDrive/stat940project'
data_path = os.path.join(parent_dir, 'data/ModelNet40')

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')

else:
    device = torch.device('cpu')
    print('running on CPU')

test_X, test_y = load_data(data_path, mode='test')
test_targets = load_targets(data_path)



running on GPU


In [ ]:
default_transforms = transforms.Compose(
    [
        ToTensor()
    ]
)

In [ ]:
testset = AdversarialDataset(test_X, test_y, test_targets, transforms=default_transforms)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
def run(targeted, file_name, output_path, model_name, signed = False):
      if model_name == "pointnet":
          model = Pointnet(device=device).to(device)
      else:
          model = DGCNN().to(device)
      checkpoint_path = os.path.join(parent_dir, f'checkpoints/{model_name}/best.pt')
      model.load_state_dict(torch.load(checkpoint_path, map_location=device)["model_state_dict"])
      model.eval()

      all_adv_pc = []
      all_gt_labels = []
      all_target_labels = []
      all_success = 0

      attacker = FGM(model, epsilon=0.2)

      for pc, labels, targets in tqdm(testloader):
          with torch.no_grad():
              pc = pc.to(device)
              labels = labels.to(device)
              targets = targets.to(device)

          if targeted:
              adv_pc, success_num = attacker.attack(pc, targets, device, targeted=targeted)
          else:
              adv_pc, success_num = attacker.attack(pc, labels, device, targeted=targeted)

          all_adv_pc += [adv_pc]
          all_gt_labels += [labels.detach().cpu().numpy()]
          all_target_labels += [targets.detach().cpu().numpy()]
          all_success += success_num
          print()

      print(f"Success Rate: {all_success / len(test_X)}")


      all_adv_pc = np.concatenate(all_adv_pc, axis=0)
      all_gt_labels = np.concatenate(all_gt_labels, axis=0)
      all_target_labels = np.concatenate(all_target_labels, axis=0)



      save_data(all_adv_pc, file_name, output_path)
      save_data(all_gt_labels, 'test_labels', output_path)
      save_data(all_adv_pc, 'target_labels', output_path)
      return all_adv_pc

In [ ]:
adv_data_path = os.path.join(parent_dir, f'data/FGM/DGCNN')
all_adv_pc, _ , _= load_adv_data(adv_data_path, targeted=False)

In [ ]:
path = os.path.join(parent_dir, 'data/FGM/Pointnet')
if not os.path.exists(path):
    os.makedirs(path)
all_adv_pc = run(False, 'adv_test_untarget', path, 'pointnet', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

41/64 successful attacks

35/64 successful attacks

38/64 successful attacks

38/64 successful attacks

38/64 successful attacks

40/64 successful attacks

38/64 successful attacks

41/64 successful attacks

42/64 successful attacks

38/64 successful attacks

43/64 successful attacks

37/64 successful attacks

38/64 successful attacks

34/64 successful attacks

31/64 successful attacks

34/64 successful attacks

34/64 successful attacks

34/64 successful attacks

40/64 successful attacks

35/64 successful attacks

41/64 successful attacks

40/64 successful attacks

35/64 successful attacks

32/64 successful attacks

28/64 successful attacks

42/64 successful attacks

41/64 successful attacks

33/64 successful attacks

35/64 successful attacks

45/64 successful attacks

35/64 successful attacks

34/64 successful attacks

39/64 successful attacks

42/64 successful attacks

35/64 successful attacks

37/64 successful attacks

38/64 successful attacks

36/64 successful attacks

17/36 succes

In [ ]:
x1, y1, z1 = all_adv_pc[1].T
x2, y2, z2 = test_X[1].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='blue')

In [ ]:
path = os.path.join(parent_dir, 'data/FGM/Pointnet')
all_adv_pc = run(True, 'adv_test_target', path, 'pointnet', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

2/64 successful attacks

0/64 successful attacks

1/64 successful attacks

4/64 successful attacks

2/64 successful attacks

2/64 successful attacks

3/64 successful attacks

2/64 successful attacks

2/64 successful attacks

2/64 successful attacks

3/64 successful attacks

3/64 successful attacks

1/64 successful attacks

2/64 successful attacks

3/64 successful attacks

2/64 successful attacks

3/64 successful attacks

4/64 successful attacks

1/64 successful attacks

1/64 successful attacks

1/64 successful attacks

2/64 successful attacks

2/64 successful attacks

3/64 successful attacks

1/64 successful attacks

2/64 successful attacks

3/64 successful attacks

1/64 successful attacks

1/64 successful attacks

3/64 successful attacks

1/64 successful attacks

2/64 successful attacks

5/64 successful attacks

1/64 successful attacks

2/64 successful attacks

1/64 successful attacks

1/64 successful attacks

3/64 successful attacks

1/36 successful attacks

Success Rate: 0.032009724

In [ ]:
x1, y1, z1 = all_adv_pc[2].T
x2, y2, z2 = test_X[2].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='red')

In [ ]:
path = os.path.join(parent_dir, 'data/FGM/DGCNN')
if not os.path.exists(path):
    os.makedirs(path)
all_adv_pc = run(False, 'adv_test_untarget', path, 'dgcnn', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

13/64 successful attacks

14/64 successful attacks

13/64 successful attacks

16/64 successful attacks

19/64 successful attacks

18/64 successful attacks

14/64 successful attacks

13/64 successful attacks

11/64 successful attacks

18/64 successful attacks

14/64 successful attacks

13/64 successful attacks

18/64 successful attacks

14/64 successful attacks

14/64 successful attacks

14/64 successful attacks

22/64 successful attacks

14/64 successful attacks

18/64 successful attacks

17/64 successful attacks

9/64 successful attacks

10/64 successful attacks

9/64 successful attacks

11/64 successful attacks

11/64 successful attacks

8/64 successful attacks

9/64 successful attacks

19/64 successful attacks

11/64 successful attacks

21/64 successful attacks

14/64 successful attacks

19/64 successful attacks

13/64 successful attacks

11/64 successful attacks

20/64 successful attacks

7/64 successful attacks

17/64 successful attacks

13/64 successful attacks

5/36 successful a

In [ ]:
x1, y1, z1 = all_adv_pc[6].T
x2, y2, z2 = test_X[6].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='blue')

In [ ]:
path = os.path.join(parent_dir, 'data/FGM/DGCNN')
all_adv_pc = run(True, 'adv_test_target', path, 'dgcnn', signed = False)

  0%|          | 0/39 [00:00<?, ?it/s]

0/64 successful attacks

0/64 successful attacks

0/64 successful attacks

1/64 successful attacks

0/64 successful attacks

0/64 successful attacks

0/64 successful attacks

0/64 successful attacks

0/64 successful attacks

1/64 successful attacks

1/64 successful attacks

0/64 successful attacks

0/64 successful attacks

1/64 successful attacks

1/64 successful attacks

2/64 successful attacks

2/64 successful attacks

0/64 successful attacks

1/64 successful attacks

0/64 successful attacks

2/64 successful attacks

0/64 successful attacks

0/64 successful attacks

0/64 successful attacks

1/64 successful attacks

0/64 successful attacks

1/64 successful attacks

1/64 successful attacks

0/64 successful attacks

1/64 successful attacks

0/64 successful attacks

0/64 successful attacks

2/64 successful attacks

0/64 successful attacks

1/64 successful attacks

0/64 successful attacks

1/64 successful attacks

2/64 successful attacks

0/36 successful attacks

Success Rate: 0.008914100

In [ ]:
x1, y1, z1 = all_adv_pc[4].T
x2, y2, z2 = test_X[4].T
pcshow(x1, y1, z1, color='red')
pcshow(x2, y2, z2, color='red')